# Solution: CNN WITHOUT Pain Survey Features

**Hypothesis**: Pain surveys cause 90% val → 60% test drop

**Key Changes**:
- ❌ **NO pain survey features** (potential label leakage)
- ✅ **Save MinMaxScaler** for consistent test preprocessing
- ✅ **Stratified split** for fair validation
- ✅ **Probability averaging** aggregation instead of majority vote
- ✅ **STRIDE=300** (no overlap) to prevent temporal leakage

This should give more honest validation F1 that matches test performance.

In [ ]:
# Set seed for reproducibility
SEED = 42

import os
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import random
import numpy as np
np.random.seed(SEED)
random.seed(SEED)

import torch
torch.manual_seed(SEED)
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
else:
    device = torch.device("cpu")

import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

In [ ]:
# Load data
df = pd.read_csv("pirate_pain_train.csv")
target = pd.read_csv("pirate_pain_train_labels.csv")

print(f"Samples: {len(df['sample_index'].unique())}")
print(f"Total frames: {len(df)}")

# Check pain survey columns
pain_survey_cols = [col for col in df.columns if 'pain_survey' in col]
print(f"\n⚠️  Found {len(pain_survey_cols)} pain survey columns: {pain_survey_cols}")
print("   These will be EXCLUDED to prevent leakage!")

In [ ]:
# Preprocess - EXCLUDING pain surveys
number_cols = ['n_legs', 'n_hands', 'n_eyes']
for col in number_cols:
    df[col] = df[col].astype('category').cat.codes

joint_cols = ["joint_" + str(i).zfill(2) for i in range(31)]
for col in joint_cols:
    df[col] = df[col].astype(np.float32)

# ✅ FIT scaler and SAVE for test time
minmax_scaler = MinMaxScaler()
df[joint_cols] = minmax_scaler.fit_transform(df[joint_cols])

with open('models/minmax_scaler.pkl', 'wb') as f:
    pickle.dump(minmax_scaler, f)
print("✅ Scaler saved to models/minmax_scaler.pkl")

# ❌ EXCLUDE pain surveys
data_cols = number_cols + joint_cols
print(f"\nUsing {len(data_cols)} features (NO pain surveys)")

# Map labels
label_mapping = {'no_pain': 0, 'low_pain': 1, 'high_pain': 2}
target['label'] = target['label'].map(label_mapping)

print("\nClass distribution:")
print(target['label'].value_counts().sort_index())

In [ ]:
# ✅ Stratified split at USER level
user_labels = target.groupby('sample_index')['label'].first()

train_users, val_users = train_test_split(
    user_labels.index, 
    test_size=0.2, 
    random_state=SEED,
    stratify=user_labels.values
)

print(f"Train users: {len(train_users)}")
print(f"Val users: {len(val_users)}")
print("\nTrain class distribution:")
print(user_labels[train_users].value_counts().sort_index())
print("\nVal class distribution:")
print(user_labels[val_users].value_counts().sort_index())

In [ ]:
# Build sequences with NO OVERLAP
WINDOW_SIZE = 300
STRIDE = 300  # ✅ No overlap!

def build_sequences(df, target_df, window=300, stride=300):
    dataset = []
    labels = []
    sample_ids = []  # Track sample ID for each sequence
    
    for id in df['sample_index'].unique():
        temp = df[df['sample_index'] == id][data_cols].values
        label = target_df[target_df['sample_index'] == id]['label'].values[0]
        
        padding_len = window - len(temp) % window if len(temp) % window != 0 else 0
        if padding_len > 0:
            padding = np.zeros((padding_len, len(data_cols)), dtype='float32')
            temp = np.concatenate((temp, padding))
        
        idx = 0
        while idx + window <= len(temp):
            dataset.append(temp[idx:idx + window])
            labels.append(label)
            sample_ids.append(id)
            idx += stride
    
    return np.array(dataset), np.array(labels), np.array(sample_ids)

df_train = df[df['sample_index'].isin(train_users)]
df_val = df[df['sample_index'].isin(val_users)]

X_train, y_train, train_sample_ids = build_sequences(df_train, target, WINDOW_SIZE, STRIDE)
X_val, y_val, val_sample_ids = build_sequences(df_val, target, WINDOW_SIZE, STRIDE)

print(f"\nTraining sequences: {X_train.shape}")
print(f"Validation sequences: {X_val.shape}")
print(f"\nWith STRIDE={STRIDE}, each sample produces ~{len(X_train)/len(train_users):.1f} sequences")

In [ ]:
# Balance dataset via oversampling
train_counts = np.bincount(y_train.astype(int))
target_count = train_counts[0]
duplication_factors = np.ceil(target_count / train_counts).astype(int)

X_train_balanced = []
y_train_balanced = []

for cls in range(len(train_counts)):
    cls_indices = np.where(y_train == cls)[0]
    for _ in range(duplication_factors[cls]):
        X_train_balanced.append(X_train[cls_indices])
        y_train_balanced.append(y_train[cls_indices])

X_train_balanced = np.concatenate(X_train_balanced, axis=0)
y_train_balanced = np.concatenate(y_train_balanced, axis=0)

# Shuffle
shuffle_idx = np.random.permutation(len(X_train_balanced))
X_train_balanced = X_train_balanced[shuffle_idx]
y_train_balanced = y_train_balanced[shuffle_idx]

print(f"Balanced training: {len(y_train_balanced)} sequences")
print("Balanced distribution:", np.bincount(y_train_balanced.astype(int)))

In [ ]:
# CNN Model
class CNN1DClassifier(nn.Module):
    def __init__(self, input_size, num_classes, dropout=0.4):
        super().__init__()
        
        self.conv1 = nn.Conv1d(input_size, 64, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(256)
        
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(dropout)
        
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = x.transpose(1, 2)
        
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.dropout(x)
        
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)
        
        x = self.relu(self.bn3(self.conv3(x)))
        
        x = self.global_pool(x).squeeze(-1)
        x = self.fc(x)
        return x

model = CNN1DClassifier(input_size=X_train.shape[-1], num_classes=3, dropout=0.4).to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Create dataloaders
BATCH_SIZE = 32

train_ds = TensorDataset(torch.from_numpy(X_train_balanced).float(), torch.from_numpy(y_train_balanced).long())
val_ds = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
scaler = torch.amp.GradScaler(enabled=(device.type == 'cuda'))

def train_epoch(model, loader, criterion, optimizer, scaler, device):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []
    
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        with torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item() * inputs.size(0)
        all_preds.extend(outputs.argmax(1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(loader.dataset)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    return avg_loss, f1

def val_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    all_probs = []
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            with torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            probs = torch.softmax(outputs, dim=1)
            
            total_loss += loss.item() * inputs.size(0)
            all_preds.extend(outputs.argmax(1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    avg_loss = total_loss / len(loader.dataset)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    return avg_loss, f1, np.array(all_preds), np.array(all_labels), np.array(all_probs)

In [ ]:
# Train
print("=" * 80)
print("Training CNN WITHOUT Pain Surveys")
print("=" * 80)

EPOCHS = 50
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(1, EPOCHS + 1):
    train_loss, train_f1 = train_epoch(model, train_loader, criterion, optimizer, scaler, device)
    val_loss, val_f1, val_preds, val_labels, val_probs = val_epoch(model, val_loader, criterion, device)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_f1 = val_f1
        patience_counter = 0
        torch.save(model.state_dict(), 'models/cnn_no_pain_surveys_best.pt')
        print(f"Epoch {epoch:3d}/{EPOCHS} | Train: Loss={train_loss:.4f}, F1={train_f1:.4f} | Val: Loss={val_loss:.4f}, F1={val_f1:.4f} ⭐")
    else:
        patience_counter += 1
        if epoch % 5 == 0:
            print(f"Epoch {epoch:3d}/{EPOCHS} | Train: Loss={train_loss:.4f}, F1={train_f1:.4f} | Val: Loss={val_loss:.4f}, F1={val_f1:.4f}")
    
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

print(f"\nBest validation F1: {best_f1:.4f}")

In [ ]:
# Final evaluation
model.load_state_dict(torch.load('models/cnn_no_pain_surveys_best.pt'))
_, val_f1, val_preds, val_labels, val_probs = val_epoch(model, val_loader, criterion, device)

print("\n" + "=" * 80)
print("📊 SEQUENCE-LEVEL RESULTS (NO Pain Surveys)")
print("=" * 80)
print(f"Validation F1: {val_f1:.4f}")

print("\n📋 Per-class metrics:")
print(classification_report(val_labels, val_preds, target_names=['no_pain', 'low_pain', 'high_pain'], digits=4))

print("\n📈 Confusion Matrix:")
cm = confusion_matrix(val_labels, val_preds)
print(cm)

In [ ]:
# ✅ SAMPLE-LEVEL EVALUATION (aggregating multiple sequences per sample)
print("\n" + "=" * 80)
print("📊 SAMPLE-LEVEL RESULTS (Aggregated Predictions)")
print("=" * 80)

# Aggregate predictions per sample using AVERAGE PROBABILITIES
sample_probs = {}
sample_true_labels = {}

for sample_id, pred, label, prob in zip(val_sample_ids, val_preds, val_labels, val_probs):
    if sample_id not in sample_probs:
        sample_probs[sample_id] = []
        sample_true_labels[sample_id] = label
    sample_probs[sample_id].append(prob)

# Average probabilities per sample
sample_predictions = {}
for sample_id, probs in sample_probs.items():
    avg_prob = np.mean(probs, axis=0)
    sample_predictions[sample_id] = np.argmax(avg_prob)

# Evaluate sample-level performance
true_labels = [sample_true_labels[sid] for sid in sorted(sample_predictions.keys())]
pred_labels = [sample_predictions[sid] for sid in sorted(sample_predictions.keys())]

sample_f1 = f1_score(true_labels, pred_labels, average='weighted', zero_division=0)

print(f"\n✅ Sample-level F1 (avg probability): {sample_f1:.4f}")
print(f"   ({len(sample_predictions)} samples)")

print("\n📋 Sample-level per-class metrics:")
print(classification_report(true_labels, pred_labels, target_names=['no_pain', 'low_pain', 'high_pain'], digits=4))

print("\n📈 Sample-level Confusion Matrix:")
cm_sample = confusion_matrix(true_labels, pred_labels)
print(cm_sample)

print("\n" + "=" * 80)
print("💡 EXPECTATION:")
print("=" * 80)
print(f"If pain surveys were causing leakage:")
print(f"  - This F1 ({sample_f1:.4f}) should be LOWER than your previous 90%")
print(f"  - But it should MATCH the test set F1 better!")
print(f"\nIf this F1 is still ~90%, pain surveys weren't the issue.")
print(f"If this F1 is ~60-70%, we found the leakage!")